# Hazard Overlay
This tutorial guides you through performing a hazard overlay with a network using RA2CE.
It assumes you already have a road network. If not, please first complete the [Network tutorial](network).

## Hazard Map
A hazard map is geospatial data representing the extent and severity of a hazard in a specific area. RA2CE supports raster formats such as GeoTIFF.

> **Note:** RA2CE is hazard agnostic: any hazard that can be represented in a raster format (flood, landslide, wildfire) can be used.

## What is hazard overlay?
The road network is represented as links and nodes, and the hazard map as a raster grid. By overlaying the network onto the hazard map, each road segment is assigned hazard values from the intersecting cells. This allows RA2CE to quantify network exposure and supports risk assessment and accessibility studies.

## Step 1. Import the Required Packages

In [ ]:
from pathlib import Path
import geopandas as gpd

from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (
    NetworkSection, NetworkConfigData, HazardSection
)
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.ra2ce_handler import Ra2ceHandler

# Root project directory
root_dir = Path(r'c:\Users\username\Documents\RA2CE_tutorial\root_dir')
network_path = root_dir / "network"
hazard_path = root_dir / "hazard"

## Step 2. Define Network and Hazard Sections
Specify the network and hazard map sections in the configuration. Provide the CRS of the hazard map and select an aggregation method for hazard values.

In [ ]:
# Define the network section
network_section = NetworkSection(
    source=SourceEnum.SHAPEFILE,
    primary_file=[network_path.joinpath("base_shapefile.shp")],
    file_id="ID",
    save_gpkg=True
)

# Define the hazard section
hazard_section = HazardSection(
    hazard_map=[hazard_path.joinpath("max_flood_depth.tif")],
    aggregate_wl=AggregateWlEnum.MEAN,
    hazard_crs="EPSG:32736",
)

# Build the full configuration
network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=root_dir.joinpath('static'),
    output_path=root_dir.joinpath('static/output_graph'),
    network=network_section,
    hazard=hazard_section
)

## Step 3. Initialize and Configure RA2CE
Generate both the base network and the overlaid hazard network. Results will be stored in `static/output_graph`.

In [ ]:
handler = Ra2ceHandler.from_config(network=network_config_data, analysis=None)
handler.configure()

## Step 4. Inspect Hazard Overlay Results
Once the analysis is complete, hazard overlay results are stored in the `output_graph` folder. Files containing `_hazard` hold the overlay results for each road segment. For example, `EV1_ma` represents the maximum flood depth for Event 1.

In [ ]:
hazard_output = root_dir / "static" / "output_graph" / "base_graph_hazard_edges.gpkg"
hazard_gdf = gpd.read_file(hazard_output, driver = "GPKG")
hazard_gdf.head()